In [ ]:
import pandas as pd
import numpy as np
import datatable as dt

# LinkImpute
Download from: http://www.cultivatingdiversity.org/software.html

Citation: https://www.g3journal.org/content/5/11/2383

To run: java -jar LinkImpute.jar -v numeric.array imputed.array

TODO: Add a small sample hapmap file for reference

# Convert hapmap file to LinkImpute numeric array and back

In [ ]:
# Load the hapmap file and remove the unused columns
df = pd.read_csv("hapmap.txt", sep="\t")
df = df.drop(df.columns[4:11], axis=1)
df.head()

In [ ]:
# Keep only allele pairs of interest
df = df[df["alleles"].isin(["A/T", "A/G", "A/C",
                            "T/A", "T/G", "T/C",
                            "G/C", "G/T", "G/A",
                            "C/A", "C/G", "C/T"])]
# To see if any allele did not pass use: "C/A", "C/G", "C/T"]) == False] 

In [ ]:
def to_numeric(row):
    row_list = row.tolist()
    # Convert the left allele to 0, right to 1, and missing to -1
    allele_dict = {row_list[1][0]: 0, row_list[1][-1]: 1, "N": -1}
    row_list[4:] = [allele_dict[allele] for allele in row_list[4:]]
    return pd.Series(row_list)

# Apply function to all rows
df = df.apply(to_numeric, axis=1)

In [ ]:
# Format data for the array format that can be used directly with the LinkImpute package
array_format = df.T
array_format = array_format.drop(array_format.index[:4])
array_format.to_csv("numeric.array", sep=" ", index=False, header=False)

In [ ]:
# Read the LinkUmpute output and put it back together with the original hapmap file 
imp = pd.read_csv("imputed.array", sep="\t", header=None)
df = pd.read_csv("hapmap.txt", sep="\t")
df = df[df["alleles"].isin(["A/T", "A/G", "A/C",
                            "T/A", "T/G", "T/C",
                            "G/C", "G/T", "G/A",
                            "C/A", "C/G", "C/T"])]
df.iloc[:,11:] = imp.T

In [ ]:
#After imputation reverse the numeric format to allelic to load hapmap file in TASSEL
def to_allelic(row):
    row_list = row.tolist()
    print(row_list)
    # Convert the left allele to 0, right to 1, and missing to -1
    allele_dict = {0: row_list[1][0], 1: row_list[1][-1]}
    row_list[11:] = [allele_dict[allele] for allele in row_list[4:]]
    return pd.Series(row_list)

# Apply function to each row to regenrate the hapmap file
df.head().apply(to_allelic, axis=1)